# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [138]:
import pandas as pd
import seaborn as sns
import numpy as np


perfil_financeiro = pd.read_csv('demo01.csv')
perfil_financeiro.info()
print(perfil_financeiro.shape)
perfil_financeiro.isna().sum()
perfil_financeiro.drop_duplicates()
print(perfil_financeiro.shape)

# Estes resultados nos mostram que não existiam dados faltantes nem linhas duplicadas em nosso dataframe...

perfil_financeiro['mau'].value_counts()

# Na nossa distribuição da variável resposta, temos 16260 bons pagadores, contra 390 maus pagadores...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16650 entries, 0 to 16649
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   16650 non-null  object 
 1   posse_de_veiculo       16650 non-null  object 
 2   posse_de_imovel        16650 non-null  object 
 3   qtd_filhos             16650 non-null  int64  
 4   tipo_renda             16650 non-null  object 
 5   educacao               16650 non-null  object 
 6   estado_civil           16650 non-null  object 
 7   tipo_residencia        16650 non-null  object 
 8   idade                  16650 non-null  float64
 9   tempo_emprego          16650 non-null  float64
 10  possui_celular         16650 non-null  int64  
 11  possui_fone_comercial  16650 non-null  int64  
 12  possui_fone            16650 non-null  int64  
 13  possui_email           16650 non-null  int64  
 14  qt_pessoas_residencia  16650 non-null  float64
 15  ma

mau
False    16260
True       390
Name: count, dtype: int64

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [140]:
data_type = perfil_financeiro.dtypes.tolist()
colunas = perfil_financeiro.columns
dataframe_tipos = pd.DataFrame([data_type], columns=colunas)
qtd_categorias = []
for c in colunas:
    qtd_categorias.append(perfil_financeiro[c].nunique())
dataframe_tipos.loc[len(dataframe_tipos)] = qtd_categorias
novos_indices = ['tipo_dado', 'Qtd_categorias']
dataframe_tipos.index = novos_indices
print(dataframe_tipos)

                  sexo posse_de_veiculo posse_de_imovel qtd_filhos tipo_renda  \
tipo_dado       object           object          object      int64     object   
Qtd_categorias       2                2               2          8          5   

               educacao estado_civil tipo_residencia    idade tempo_emprego  \
tipo_dado        object       object          object  float64       float64   
Qtd_categorias        5            5               6     5298          3005   

               possui_celular possui_fone_comercial possui_fone possui_email  \
tipo_dado               int64                 int64       int64        int64   
Qtd_categorias              1                     2           2            2   

               qt_pessoas_residencia   mau  
tipo_dado                    float64  bool  
Qtd_categorias                     9     2  


Algo a se destacar aqui, é que as variáveis do tipo float, por exemplo, nã são muito apropriadas ao uso de categorias, portanto a nossa 'quantidade de categorias' no que diz respeito a elas, não é muito relevante.

#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [143]:
colunas_purificadas = list(colunas)
print(colunas_purificadas)
del colunas_purificadas[8]
del colunas_purificadas[8]

for col in colunas_purificadas:
    dummies = None
    dummies = pd.get_dummies(perfil_financeiro[col], prefix = col)
    perfil_financeiro = pd.concat([perfil_financeiro, dummies], axis = 1)

print(perfil_financeiro)

['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego', 'possui_celular', 'possui_fone_comercial', 'possui_fone', 'possui_email', 'qt_pessoas_residencia', 'mau']
      sexo posse_de_veiculo posse_de_imovel  qtd_filhos            tipo_renda  \
0        M                Y               Y           0               Working   
1        F                N               Y           0  Commercial associate   
2        F                N               Y           0  Commercial associate   
3        M                Y               Y           0               Working   
4        F                Y               N           0               Working   
...    ...              ...             ...         ...                   ...   
16645    F                N               Y           0               Working   
16646    F                N               Y           0  Commercial associate   
16647    M      

Com isso, já criamos os dummies para as variáveis apropriadas. Resta, agora, exluir nossas variáveis que deram origem aos nossos dummies, e também as duas variáveis que não são apropriadas para a biblioteca scikitlearn, isto é, as mesmas duas que tiramos ao criarmos a "colunas_filtradas". Mas também iremos tirar todas as variáveis que deram origem aos nossos dummies, ou seja, iremos excluir basicamente tudo, exceto a nossa variável default, pois é conveniente ter ela conoco ainda. Na verdade, parece completamente redundante também ter uma coluna para mau_False e outra para mau_True, pois são completamente redundantes. Então:

In [193]:
perfil_financeiro_preparado = perfil_financeiro.drop(columns = colunas, inplace = False)
print(perfil_financeiro_preparado)

# As nossas variáveis que eram binárias também, por consequência, estão poluídas, pois se uma for True, a outra necessariamente será False, e a
# existencia de duas colunas é reduntante. Consultemos nosso dataframe de tipagem:
is_binary = dataframe_tipos.map(lambda x: x == 2)
colunas_binarias = is_binary.columns[is_binary.any()].tolist()
for i in range(0, len(colunas_binarias)):
    colunas_binarias[i] += '_False'
print(colunas_binarias)
perfil_financeiro_preparado.drop(columns = ['mau_False'], inplace = True)


# Deixo aqui um comentário importante: seria extremamente desejável eliminar uma das dummies que foram criadas em variáveis binárias complementaros,
# , mas não consegui um método eficiente de fazer isso. Até criei um método de ver quais eram as colunas binárias, mas o fato de ser não uniforme a
# nomenclatura delas impossibilitou a resolução prática. Então, como é apenas um exercício, e estes dados, eu suponho, não serão exibidos em um contexto
# onde a performance de execução seja indispensável, deixarei a atividade deste modo... no entanto, consciente de que não é o ideal... 

       sexo_F  sexo_M  posse_de_veiculo_N  posse_de_veiculo_Y  \
0       False    True               False                True   
1        True   False                True               False   
2        True   False                True               False   
3       False    True               False                True   
4        True   False               False                True   
...       ...     ...                 ...                 ...   
16645    True   False                True               False   
16646    True   False                True               False   
16647   False    True               False                True   
16648   False    True               False                True   
16649    True   False                True               False   

       posse_de_imovel_N  posse_de_imovel_Y  qtd_filhos_0  qtd_filhos_1  \
0                  False               True          True         False   
1                  False               True          True         Fal

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [243]:
print(perfil_financeiro_preparado.columns)
tabela = pd.crosstab(perfil_financeiro_preparado['possui_email_1'], perfil_financeiro_preparado['posse_de_veiculo_Y'])
tabela_soma_default = pd.crosstab(perfil_financeiro_preparado['possui_email_1'], perfil_financeiro_preparado['posse_de_veiculo_Y'],
            values = perfil_financeiro_preparado['mau_True'], aggfunc='sum'
           )
# prop_carros = soma_possui_carro/(soma_possui_carro+soma_nao_carro)
# prop_email = soma_possui_email/(soma_nao_email+soma_possui_email)
# print(prop_carros)
# print(prop_email)
print(tabela)
print(tabela_soma_default)

Index(['sexo_F', 'sexo_M', 'posse_de_veiculo_N', 'posse_de_veiculo_Y',
       'posse_de_imovel_N', 'posse_de_imovel_Y', 'qtd_filhos_0',
       'qtd_filhos_1', 'qtd_filhos_2', 'qtd_filhos_3', 'qtd_filhos_4',
       'qtd_filhos_5', 'qtd_filhos_7', 'qtd_filhos_14',
       'tipo_renda_Commercial associate', 'tipo_renda_Pensioner',
       'tipo_renda_State servant', 'tipo_renda_Student', 'tipo_renda_Working',
       'educacao_Academic degree', 'educacao_Higher education',
       'educacao_Incomplete higher', 'educacao_Lower secondary',
       'educacao_Secondary / secondary special', 'estado_civil_Civil marriage',
       'estado_civil_Married', 'estado_civil_Separated',
       'estado_civil_Single / not married', 'estado_civil_Widow',
       'tipo_residencia_Co-op apartment', 'tipo_residencia_House / apartment',
       'tipo_residencia_Municipal apartment',
       'tipo_residencia_Office apartment', 'tipo_residencia_Rented apartment',
       'tipo_residencia_With parents', 'possui_celular_1

A enorme discrepancia entre a proporcionalidedade de posse de veículo, quando a quantidade de não possuintes de veículo quase dobrou o valor de não possuintes, na linha de não possuidores de email, a quantidade de defaults manteve seu crescimento aproximadamente na mesma proporção. Agora, quando a 
quantidade de possuidores de email diminui em aproximadamente 25%, a alteração no default é mínima... então, a variável de posse de veículo parece acompanhar o default com mais eficiência.

#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [247]:
perfil_financeiro_preparado.to_csv('financeiro_trabalhado', index=False)